In [2]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

type constant = Constant of string


type variable = Variable of string


type func = Function of string * term list
and term = C of constant | V of variable | F of func


type predicate = Predicate of string * term list


type head = predicate


type body = predicate list


type clause = Fact of head | Rule of head * body


type goal = predicate list


type program = clause list


In [3]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

val string_of_f_list : ('a -> string) -> 'a list -> string = <fun>


val string_of_term : term -> string = <fun>


val string_of_predicate : predicate -> string = <fun>


val string_of_predicate_list : predicate list -> string = <fun>


val string_of_goal : predicate list -> string = <fun>


val string_of_clause : clause -> string = <fun>


val string_of_program : clause list -> string = <fun>


Below are more helper functions:

In [4]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

val var : string -> variable = <fun>


val var_t : string -> term = <fun>


val const : string -> constant = <fun>


val const_t : string -> term = <fun>


val func : string -> term list -> func = <fun>


val func_t : string -> term list -> term = <fun>


val pred : string -> term list -> predicate = <fun>


val fact : head -> clause = <fun>


val rule : head -> body -> clause = <fun>


In [5]:
let rec occurs_check v t =
(
    match t with
    | C (Constant const_term) -> false
    | V (Variable var_term) -> 
    (
        match v with 
        | Variable x -> (x=var_term)
    )
    | F (Function (f,tl)) -> (ochelper v tl)
)

and ochelper v tl = 
(
    match tl with
    | [] -> false
    | h::tail -> if (occurs_check v h) then true else (ochelper v tail)
)


val occurs_check : variable -> term -> bool = <fun>
val ochelper : variable -> term list -> bool = <fun>


In [7]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* DO NOT CHANGE THE ABOVE DEFINITION 
   You can ignore any alerts that you may get regarding the usage of Pervasives.*)
   
   
let rec vot_helper t curSet = 
(
    match t with
    | C const_term -> curSet
    | V var_term -> VarSet.add var_term curSet
    | F (Function (f,tl)) -> (tl_helper tl curSet)
)
and tl_helper tl curSet = 
(
    match tl with
    | [] -> curSet
    | h::tail -> tl_helper tail (vot_helper h curSet)
)

let variables_of_term t = vot_helper t VarSet.empty




let variables_of_predicate (Predicate (_, l)) = tl_helper l VarSet.empty



let rec voc_helper c curSet = 
(
    match c with
    | Fact f -> VarSet.union curSet (variables_of_predicate f)
    | Rule (h,b) -> (pl_helper b (VarSet.union curSet (variables_of_predicate h)))
)

and pl_helper pl curSet = 
(
    match pl with
    | [] -> curSet
    | h::tail -> pl_helper tail (VarSet.union curSet (variables_of_predicate h) )
)
  
let variables_of_clause c = voc_helper c VarSet.empty



File "[7]", line 1, characters 64-82:
1 | module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                    ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module VarSet :
  sig
    type elt = variable
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val disjoint : t -> t -> bool
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    

val vot_helper : term -> VarSet.t -> VarSet.t = <fun>
val tl_helper : term list -> VarSet.t -> VarSet.t = <fun>


val variables_of_term : term -> VarSet.t = <fun>


val variables_of_predicate : predicate -> VarSet.t = <fun>


val voc_helper : clause -> VarSet.t -> VarSet.t = <fun>
val pl_helper : body -> VarSet.t -> VarSet.t = <fun>


val variables_of_clause : clause -> VarSet.t = <fun>


In [9]:
module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* DO NOT CHANGE THE ABOVE DEFINITION 
   You can ignore any alerts that you may get regarding the usage of Pervasives.*)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"

let rec substitute_in_term s t =
(
    match t with
    | C const_term -> t
    | V (Variable var_term) -> 
    (
        match (Substitution.find_opt (Variable var_term) s) with
        | None -> t
        | Some x -> x
    )
    | F (Function (f,tl)) -> (func_t f (tl_helper tl s)) 
)
and tl_helper tl s =
(
    match tl with
    | [] -> []
    | h::tail -> (substitute_in_term s h)::(tl_helper tail s)
)

let rec substitute_in_predicate s (Predicate (p,l)) = (Predicate (p,(tl_helper l s)))

  
let rec substitute_in_clause s c =

(
    match c with
    | Fact f -> (fact (substitute_in_predicate s f))
    | Rule (h,b) -> (rule (substitute_in_predicate s h) (pl_helper2 b s))
)

and pl_helper2 pl s =
(
    match pl with
    | [] -> []
    | h::tail -> (substitute_in_predicate s h)::(pl_helper2 tail s)
)

File "[9]", line 1, characters 70-88:
1 | module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                          ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module Substitution :
  sig
    type key = variable
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a 

val string_of_substitution : term Substitution.t -> string = <fun>


val substitute_in_term : term Substitution.t -> term -> term = <fun>
val tl_helper : term list -> term Substitution.t -> term list = <fun>


val substitute_in_predicate : term Substitution.t -> predicate -> predicate =
  <fun>


val substitute_in_clause : term Substitution.t -> clause -> clause = <fun>
val pl_helper2 : body -> term Substitution.t -> body = <fun>


We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [11]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

val counter : int ref = {contents = 0}


val fresh : unit -> term = <fun>


val freshen : clause -> clause = <fun>


For example, 

In [12]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

val c : clause =
  Rule
   (Predicate ("p", [V (Variable "X"); V (Variable "Y"); C (Constant "a")]),
   [Predicate ("q", [V (Variable "X"); C (Constant "b"); C (Constant "a")])])


- : string = "p(X, Y, a) :- q(X, b, a)."


- : string = "p(_G0, _G1, a) :- q(_G0, b, a)."


## Problem 4

Implement the function:

```ocaml
compose: term Substitution.t -> term Substitution.t -> term Substitution.t
```

that composes two substitutions. Use the precise definition of composition from the lectures.


In [13]:
let same_term t1 t2 = 
(
    match t1 with
    | V (Variable v1) ->
    (
        match t2 with
        | V (Variable v2) -> (v1=v2)
        | _ -> false
    )
    | _ -> false
)

let rec map_sub l s2 = 
(
    match l with
    | [] -> []
    | (hkey, hval)::tail ->
    (
        let newVal = (substitute_in_term s2 hval) in
        if (same_term (V(hkey)) newVal) then (map_sub tail s2)
        else (hkey, newVal)::(map_sub tail s2)
    )
)

let rec yput s1 l2 =
(
    match l2 with
    | [] -> s1
    | (hkey, hval)::tail ->
    (
        match (Substitution.find_opt hkey s1) with
        | None -> yput (Substitution.add hkey hval s1) tail
        | Some _ -> yput s1 tail
    )
)

let rec l2m l m=
(
    match l with
    | [] -> m
    | (hkey, hval)::tail -> (l2m tail (Substitution.add hkey hval m))
)

let compose s1 s2 = 
(
    let l1 = (Substitution.bindings s1) in
    let ret = (l2m (map_sub l1 s2) Substitution.empty) in
    let l2 = (Substitution.bindings s2) in
    (yput ret l2)
)



val same_term : term -> term -> bool = <fun>


val map_sub :
  (variable * term) list -> term Substitution.t -> (variable * term) list =
  <fun>


val yput :
  'a Substitution.t -> (Substitution.key * 'a) list -> 'a Substitution.t =
  <fun>


val l2m :
  (Substitution.key * 'a) list -> 'a Substitution.t -> 'a Substitution.t =
  <fun>


val compose :
  term Substitution.t -> term Substitution.t -> term Substitution.t = <fun>


In [15]:

let rec mgu_helper xtemp ytemp theta =
(
let x = (substitute_in_term theta xtemp) in
let y = (substitute_in_term theta ytemp) in
(
    match x with
    | V (vx) -> 
    (
        if ( not (occurs_check vx y)) then Some (compose theta (Substitution.add vx y (Substitution.empty)))
        else
        (
            if(same_term x y) then Some (theta)
            else None
        )
    )
    | C (cx) -> 
    (
        match y with
        | V (vy) -> Some(compose theta (Substitution.add vy x (Substitution.empty)))
        | C (cy) -> if(cx=cy) then Some(theta) else None
        | _ -> None
    )
    | F (Function (fx,tlx)) ->
    (
        match y with
        | F (Function (fy,tly)) -> 
            if (not (fx=fy)) then None
            else
            (
                if (not ((List.length tlx) = (List.length tly))) then None
                else (mgu_fhelper tlx tly theta)
            )
        | V (vy) -> if ( not (occurs_check vy x)) then Some (compose theta (Substitution.add vy x (Substitution.empty)))
            else None
        | C (cy)-> None
    )
)
)
and mgu_fhelper lx ly theta =
(
    match lx with
    | [] -> Some (theta)
    | x::tx ->
    (
        match ly with
        | [] -> None
        | y::ty -> 
        (
            match (mgu_helper x y theta) with
            | None -> None
            | Some newTheta -> (mgu_fhelper tx ty newTheta)
        )
    )
)

let mgu t1 t2 = (mgu_helper t1 t2 (Substitution.empty))

let mgu_predicate (Predicate (p1,tlx)) (Predicate (p2,tly)) =
(
    if (not (p1=p2)) then None
    else
    (
        if (not ((List.length tlx) = (List.length tly))) then None
        else (mgu_fhelper tlx tly (Substitution.empty))
    )
)

val mgu_helper :
  term -> term -> term Substitution.t -> term Substitution.t option = <fun>
val mgu_fhelper :
  term list -> term list -> term Substitution.t -> term Substitution.t option =
  <fun>


val mgu : term -> term -> term Substitution.t option = <fun>


val mgu_predicate : predicate -> predicate -> term Substitution.t option =
  <fun>


In [18]:
let rec append list1 list2 =
(
    match list1 with
    | [] -> list2
    | head :: tail -> (head :: (append tail list2))
)
  
let rec sub_in_list theta l =
(
    match l with 
    | [] -> []
    | h::t -> (substitute_in_predicate theta h)::(sub_in_list theta t)
)


let rec chelper goal resolvant program acc h b =
(
        match resolvant with
        | cur_res::res_rest ->
        (
            match (mgu_predicate cur_res h) with
            | None -> acc
            | Some theta -> (append acc (qhelper program (sub_in_list theta goal) (sub_in_list theta (append b res_rest))))
        )
)
  
and fhelper goal resolvant program acc pclause =
(
    match (freshen pclause) with
    | Fact h -> (chelper goal resolvant program acc h [])
    | Rule (h, b) -> (chelper goal resolvant program acc h b)
)

and lfhelper acc program goal resolvant =
(
    
    List.fold_left (fhelper goal resolvant program) [] program
)

and qhelper program goal resolvant =
(
    if ((List.length resolvant)=0) then [goal]
    else (lfhelper [] program goal resolvant)
)



let query program goal = (qhelper program goal goal)

val append : 'a list -> 'a list -> 'a list = <fun>


val sub_in_list : term Substitution.t -> predicate list -> predicate list =
  <fun>


File "[18]", lines 18-26, characters 0-1:
18 | (
19 |         match resolvant with
20 |         | cur_res::res_rest ->
21 |         (
22 |             match (mgu_predicate cur_res h) with
23 |             | None -> acc
24 |             | Some theta -> (append acc (qhelper program (sub_in_list theta goal) (sub_in_list theta (append b res_rest))))
25 |         )
26 | )
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
[]


val chelper :
  predicate list ->
  predicate list ->
  clause list -> predicate list list -> head -> body -> predicate list list =
  <fun>
val fhelper :
  predicate list ->
  predicate list ->
  clause list -> predicate list list -> clause -> predicate list list = <fun>
val lfhelper :
  'a list ->
  clause list -> predicate list -> predicate list -> predicate list list =
  <fun>
val qhelper :
  clause list -> predicate list -> predicate list -> predicate list list =
  <fun>


val query : clause list -> predicate list -> predicate list list = <fun>
